In [2]:
import sys
sys.path.append("..")   # this allows us to import from sibling directory

from code_to_import.dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from code_to_import.Classifier_module import Classifier
from torch.autograd import Variable


In [3]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
three_layers = False

path_to_model = "../models/two_layers/twoLayerModel_weights_gpu_mumu0"

event_type = "mumu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [151]:
if not three_layers:
    # model = 0
    model = Classifier()
    model.load_state_dict(torch.load(path_to_model))
#     model.eval()

In [152]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()

['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [153]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor).requires_grad_(True))

In [154]:
input.requires_grad=True

In [155]:
# optimizer = torch.optim.Adam(model.parameters(), , betas=(opt.b1, opt.b2)) 
# criterion = nn.MSELoss()   # mean squared error loss
criterion = nn.CrossEntropyLoss()


In [156]:
# model.retain_grad()
input.retain_grad()
output = model(input).requires_grad_(True)
output.retain_grad()

loss = criterion(output, target)
loss.retain_grad()
print(loss)

# external_grad = torch.tensor(1.0)
loss.backward()

tensor(0.8287, grad_fn=<NllLossBackward>)


In [157]:
print(input.grad)

tensor([[-5.6182e-06, -4.6110e-05, -7.6437e-05,  ..., -4.3662e-05,
         -2.0285e-04,  2.2462e-04],
        [-6.7044e-04,  3.5851e-04, -2.2236e-04,  ...,  1.9437e-03,
          2.3353e-04, -1.2688e-03],
        [ 4.1863e-05,  4.1750e-04,  1.9946e-05,  ..., -1.5545e-03,
          1.2086e-03,  5.1811e-03],
        ...,
        [ 1.1768e-04, -4.3671e-04,  3.2609e-04,  ..., -2.4839e-04,
         -1.8898e-04, -1.1107e-03],
        [-2.6824e-04,  2.7408e-04,  5.0225e-04,  ...,  1.2044e-03,
          6.4162e-04, -1.4682e-03],
        [-1.3703e-04, -4.9907e-05, -4.1700e-04,  ..., -1.0898e-03,
          3.0540e-03,  1.3382e-03]])


In [158]:
mean_grad = input.grad.detach().numpy().mean(axis=0)
max_grad = np.max(mean_grad)

mean_grad = mean_grad/max_grad
mean_grad

array([-0.06997979,  0.02594084,  0.11939513,  0.37060305,  0.141562  ,
       -0.01750993, -0.43100595, -0.00697539,  0.35383692,  0.13255574,
       -0.12089994, -0.19174351,  0.02859057, -0.00628711,  0.04302566,
       -0.06736392,  0.14468536, -0.10258827, -0.0386091 ,  0.03800865,
        0.02601019,  0.0757686 ,  0.12218693,  0.04713289, -0.13994537,
        0.03265835,  0.21447216, -0.23931028,  0.07057077, -0.04555491,
        0.1129856 ,  0.13698617, -0.32082236,  0.09326252, -0.23700412,
       -0.14361113, -0.06026587, -0.04251185, -0.06745239, -0.18308595,
       -0.04266756, -0.01090007, -0.0108495 ,  0.1654187 , -0.07310255,
        0.09203216, -0.17530386,  0.00456248, -0.14320344, -0.08869706,
       -0.107295  , -0.03907412,  0.10944227, -0.01416151, -0.10861064,
       -0.05098709,  0.13560188, -0.04293071, -0.06636702,  0.14060594,
       -0.04450048, -0.07949169,  0.11944798, -0.10731756, -0.24334773,
        0.16037345,  0.04992399,  0.28404215, -0.04605536,  0.06

In [159]:
std_grad = np.std(input.grad.detach().numpy(), axis=0)/np.sqrt(input.shape[0])
std_grad

array([3.9098800e-06, 4.3863274e-06, 5.0541457e-06, 5.6333165e-06,
       3.8558601e-06, 4.1174508e-06, 5.7491097e-06, 4.2558377e-06,
       5.4581014e-06, 4.8926386e-06, 5.2894688e-06, 5.6469626e-06,
       3.2077639e-06, 3.2904959e-06, 3.3568510e-06, 3.2814362e-06,
       3.9483725e-06, 3.2661096e-06, 3.3889648e-06, 3.1269274e-06,
       3.6886715e-06, 3.4909363e-06, 3.2967882e-06, 3.5786743e-06,
       4.8561346e-06, 4.3178106e-06, 4.9110772e-06, 5.4638749e-06,
       3.6364954e-06, 3.4914940e-06, 3.7663374e-06, 3.6925164e-06,
       2.4913695e-06, 3.8963603e-06, 2.8693416e-06, 2.8240972e-06,
       5.3635631e-06, 4.2372030e-06, 3.9604761e-06, 3.9575384e-06,
       5.1324159e-06, 2.6691011e-06, 2.9818161e-06, 2.8218744e-06,
       3.0187014e-06, 2.6084483e-06, 3.1425036e-06, 2.6318551e-06,
       2.6347420e-06, 2.7328949e-06, 3.0207113e-06, 3.0470444e-06,
       2.9773487e-06, 2.9035227e-06, 2.7017911e-06, 3.1662530e-06,
       3.0328561e-06, 3.3056415e-06, 2.9220764e-06, 3.2897126e

In [160]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [161]:
for i in mean_grad.argsort()[::-1]:
#     if mean_grad[i] - 2 * r.importances_std[i] > 0:
    print(f"{input_variables[i]:<8}                    "
          f"{mean_grad[i]:.3f}"
          f" +/- {std_grad[i]:.3f}")


nubar_pt                    1.000 +/- 0.000
ttbar_mass                    0.431 +/- 0.000
lnubar_delta_eta                    0.371 +/- 0.000
bnu_delta_eta                    0.354 +/- 0.000
top_pt                      0.284 +/- 0.000
ttbar_boosted_pt                    0.264 +/- 0.000
wminusb_delta_eta                    0.214 +/- 0.000
wminus_pt                    0.212 +/- 0.000
ttbar_phi                    0.165 +/- 0.000
wplus_phi                    0.160 +/- 0.000
lbarb_delta_phi                    0.145 +/- 0.000
lbarb_delta_eta                    0.142 +/- 0.000
nu_eta                      0.141 +/- 0.000
wminusbbar_delta_phi                    0.137 +/- 0.000
bbar_eta                    0.136 +/- 0.000
bnubar_delta_eta                    0.133 +/- 0.000
bbarnu_delta_phi                    0.122 +/- 0.000
nubar_phi                    0.119 +/- 0.000
lnu_delta_eta                    0.119 +/- 0.000
wminusb_delta_phi                    0.113 +/- 0.000
lbar_eta                    